<h1>Mean-Variance Optimization - Analysis</h1>
<h4>By Peter Hindi</h4>	
<p>In this notebook, we use our ML portfolio-optimization models defined in the underlying functions notebook to find a set of optimal portfolios and plot them to create an efficient frontier. This will require us to introduce our own data for expected returns of each asset class in our universe and the expected covariance between them. For this example, we use an emerging market index, an index of the S&P 500, and bitcoin returns as our asset classes</p>

***

In [7]:
#Import packages
using JuMP, Pkg, Ipopt, Plots, CSV, DataFrames, Statistics, NBInclude

#Connect to .ipynb file with functions created for our use
include("Src/Optimization Functions.jl")

<h5>Reading in our Data</h5>
<p>Here, we read in our returns data for the three asset classes in the universe from a CSV file. We then assign each asset class' returns into an associated vector for use in calculating our mean return vector and covariance matrix.</p>

***

In [3]:
#Read in our csv file with asset-class level returns as a dataframe
returndf = CSV.read("RA Sample Model Data.csv", DataFrame)

#Assign returns for each asset class into a respective vector with appropriate title
em_returns = vec(returndf[!, Symbol("Emerging Market Index")])
sp500_returns = vec(returndf[!, Symbol("S&P 500")])
bitcoin_returns = vec(returndf.Bitcoin);

<h5>Creating our Mean-return Vector and Covariance Matrix</h5>
<p>Here, we use our returns data for each asset class to create an expected return vector, a vector with the mean return (our assumption is that mean return per asset class is equal to its expected return) for each asset class in vector form. We will be able to multiply the vector transpose by our decision variables to achieve a weighted average, or the expected return of our portfolio.</p>

We then create our covariance matrix of returns for each asset class, which is a diagonally-symmetric matrix that describes the covariance in returns between each asset class. This is important for computing the variance of our portfolio and introduces the quadratic element of our problem. The variance of our portfolio will be the weight vector transpose multiplied by the covariance matrix multiplied by the weight vector again.

***

In [3]:
#Create a single vector of asset class mean returns to be used as a model input
mean_returns = [mean(em_returns),mean(sp500_returns),mean(bitcoin_returns)]

#Create a covariance matrix as an input for computing portfolio-level variance in the model   
var_matrix = hcat(em_returns, sp500_returns, bitcoin_returns)
cov_matrix = cov(var_matrix);

3×3 Matrix{Float64}:
 0.00208889  0.0015004   0.00245769
 0.0015004   0.00160967  0.00225019
 0.00245769  0.00225019  0.0269793

<h5>Testing our Models</h5>
<p>Here, we run both of our optimization models a single time and print the results. In the first run, we aim to minimize the risk (variance) of our portfolio returns while maintaining a required return of 3%. In the second run, we aim to maximize returns given a risk (variance) tolerance of 1%. Outputted is the return and variance for each optimal portfolio.</p>

***

In [4]:
#We test our mean-variance optimization models by calling the function
minvariance=model_min_variance(mean_returns, cov_matrix, 0.03)
maxreturn=model_max_return(mean_returns, cov_matrix, 0.01)

#We print the results 
print("Min Variance Model Return: " * string(minvariance[1]) * ", Variance:" * string(minvariance[2]) *"\n")
print("Max Return Model Return: " * string(maxreturn[1]) * ", Variance:" * string(maxreturn[2]) *"\n")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Min Variance Model Return: 0.029999995760837818, Variance:0.0025575689997543415
Max Return Model Return: 0.05386998332446771, Variance:0.010000008834839981


<h5>Building our Efficient Frontier</h5>
<p>Here, we first want to create a sample of optimal portfolios across different levels of required returns for the minimum variance model and across different risk tolerances for the return maximization model. We do this by calling our "model_iterator" function with our mean returns vector and covariance matrix for each model and provide our minimum and maximum levels of the associated required return/risk tolerance with a desired increment between each point. The output of the iterator function will be two vectors with the return and variance of each portfolio tested following the constraint input, contained in a tuple.

We then call our "make_efficient_frontier" function with the risk and return vectors from the model. We run the function twice to create two frontiers for each of our models and compare them. The function plots each portfolio generated by our model on a set of risk (x) and return (y) axes to visually represent our optimal portfolios at each level of risk and return. As we can see, the scatterplots are virtually the same (they are- with the only difference being the sample size and range of constraints tested). We can also see that as variance increases, the additional return we get from an equal increase in risk begins to diminish. This relationship is non-linear, and indeed quadratic.

The shape of the curve suggests that an investor with a very low-risk portfolio could benefit dramatically by taking on some additional risk through higher returns, while an investor taking on excessive amounts of risk will experience diminishing returns and the additional return is likely not worth the trade-off at some point. As well, when you are very exposed to a risky asset class, it seems that taking on less risk will not cause a big decrease in returns to an extent. Given a moderately risk-averse temperament, it is likely that most of us would want a portfolio somewhere on the midpoint.</p>

***

In [5]:
#Call our model iterator function to run our variance and return-optimizing models
finalminvar = model_iterator(model_min_variance, mean_returns, cov_matrix, 0.01, 0.08, 0.0025)
finalmaxret = model_iterator(model_max_return, mean_returns, cov_matrix, 0.0015, 0.026, 0.001)

#Use our resulting data from the iterator function to build efficient frontier scatter plots
make_efficient_frontier(finalminvar[1],finalminvar[2])
make_efficient_frontier(finalmaxret[1],finalmaxret[2])

GKS: dlopen(/Users/peterhindi/.julia/artifacts/929d6892d4c6cbd69aa67a767339981456f74a44/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/peterhindi/.julia/artifacts/929d6892d4c6cbd69aa67a767339981456f74a44/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0
